# **GRADED CHALLENGE 3 :**

## **I. Perkenalan :**


Nama : Muhammad Farhan Darmawan

Batch : 019

## **Problem Statement :**

Tim Management dari The Look ingin mengetahui peforma dari penjualan pada platformnya selama tahun 2022 dari Bulan Januari-September di Amerika berdasarkan jumlah transaksi, user, dan total pendapatan yang didapat  mengalami peningkatan atau penurunan. Nantinya peforma ini akan dijadikan acuan untuk tim management agar dapat memberikan pelayanan terbaik bagi user yang menggunakan platform e-commerce The Look.

## **SMART METHOD :**

`Spesific` : Mengetahui kegiatan penjualan yang terjadi di The Look selama tahun 2022 dari Bulan Januari sampai September

`Measurable` : Menggunakan data jumlah transaksi, user, dan total pendapatan sebagai acuan untuk melihat peforma dari The Look

`Achievable` : Pengolahan data menggunakan metode data wrangling atau preprocessing untuk menampilkan beberapa insight apakah mengalami peningkatan atau penurunan

`Relevant` : Tim management tentunya akan mendapatkan insight terkait peforma yang didapat dan dapat dijadikan evaluasi untuk meningkatkan dan memberikan pelayanan terbaik pada Bulan Oktober-Desember tahun 2022

`Time-Bond` : Pengunaan data yang dibutuhkan yakni Tahun 2022 dari Bulan Januari-September

## **Objective :**

The Look merupakan salah satu platform e-commerce terbesar. Sebagai seorang data science saya ingin mencoba melakukan data wrangling/preprocessing dari data penjualan yang dimiliki The Look.

Disini saya akan me-preprocessing data menggunakan DBMS SQL namun saya gunakan pada bahasa python.



## **II. Import Library**

In [1]:
import pandas as pd
from google.cloud import bigquery

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = "sql-ds-batch-019" #Nama project pada Bigquery
client = bigquery.Client(project=project_id)

Authenticated


## **III. Jawaban Penjabaran**

**1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?**

In [3]:
# Memasukan data_jumlah_transaksi menggunakan quey SQL
data_jumlah_transaksi = client.query('''
SELECT 
  EXTRACT(year FROM created_at) AS year, # menggambil data tahun
  EXTRACT(month FROM created_at) AS month, #mengambil data bulan
  EXTRACT(QUARTER FROM created_at) AS quartile_month, #mengambil data quartil 
  COUNT(order_id) AS jumlah_transaksi
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE status = 'Complete' AND EXTRACT(year FROM created_at) = 2022 AND EXTRACT(QUARTER FROM created_at) BETWEEN 1 AND 3
GROUP BY year,month, quartile_month
ORDER BY month ASC;
''').to_dataframe()

In [4]:
# Menampilkan data_jumlah_transaksi
data_jumlah_transaksi

,year,month,quartile_month,jumlah_transaksi
0,2022,1,1,742
1,2022,2,1,720
2,2022,3,1,881
3,2022,4,2,856
4,2022,5,2,949
5,2022,6,2,924
6,2022,7,3,1032
7,2022,8,3,1140
8,2022,9,3,1147


Insight :

- Diketahui dari data diatas bahwa berdasarkan jumlah transaksi di The Look, dari Bulan Januari-September mengalami peningkatan pembelian pada platform The Look

**2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?**

In [5]:
# Memasukan data_total_USD menggunakan query SQL
data_total_USD = client.query('''
SELECT 
  EXTRACT(year FROM created_at) AS year,
  EXTRACT(month FROM created_at) AS month,
  EXTRACT(QUARTER FROM created_at) AS quartile_month,
  SUM(sale_price) AS price
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status = 'Complete' AND EXTRACT(year FROM created_at) = 2022
AND EXTRACT(QUARTER FROM created_at) BETWEEN 1 AND 3
GROUP BY year,month, quartile_month
ORDER BY month ASC;
''').to_dataframe()

In [6]:
# Menampilkan data_total_USD
data_total_USD

,year,month,quartile_month,price
0,2022,1,1,67617.650007
1,2022,2,1,62862.360057
2,2022,3,1,74691.140037
3,2022,4,2,71157.740146
4,2022,5,2,80874.800056
5,2022,6,2,86017.100107
6,2022,7,3,89794.310054
7,2022,8,3,98879.500059
8,2022,9,3,101128.680143


Insight :

- Diketahui dari data diatas bahwa total revenue yang didapatkan dari platform The Look tiap bulannya mengalami peningkatan

**3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?**

In [7]:
# Memasukan data_jumlah_user menggunakan query SQL
data_jumlah_user = client.query('''
SELECT 
  EXTRACT(year FROM created_at) AS year,
  EXTRACT(month FROM created_at) AS month,
  EXTRACT(QUARTER FROM created_at) AS quartile_month,
  COUNT(DISTINCT user_id) AS jumlah_user
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
ON a.product_id = b.id
WHERE status = 'Complete' AND EXTRACT(year FROM created_at) = 2022
AND EXTRACT(QUARTER FROM created_at) BETWEEN 1 AND 3
GROUP BY year,month,quartile_month
ORDER BY month ASC;
''').to_dataframe()

In [8]:
# Menampilkan data_jumlah_user
data_jumlah_user

,year,month,quartile_month,jumlah_user
0,2022,1,1,761
1,2022,2,1,732
2,2022,3,1,900
3,2022,4,2,865
4,2022,5,2,958
5,2022,6,2,936
6,2022,7,3,1044
7,2022,8,3,1153
8,2022,9,3,1158


Insight :

- Dari data diatas diketahui bahwa berdasarkan jumlah user, semakin mendekati kuartal 4 jumlah user mengalami peningkatan

- Dari ketiga data yakni jumlah transaksi, total revenue(price), dan jumlah user ketiganya mengalami peningkatan pada saat mendekati kuartal 4

**4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!**

In [9]:
# Memasukan data_jumlah_category menggunakan query SQL
data_jumlah_category = client.query('''
SELECT year, month,quartile_month, jenis_category, jumlah_category
FROM(
      SELECT year, month, quartile_month,jenis_category,
      RANK() OVER(PARTITION BY month ORDER BY jumlah_category DESC) AS rnk, jumlah_category #menggunakan windows function rank()
      FROM( SELECT category AS jenis_category, 
            EXTRACT(year FROM created_at) AS year,
            EXTRACT(month FROM created_at) AS month,
            EXTRACT(QUARTER FROM created_at) AS quartile_month, 
            COUNT(category) AS jumlah_category
            FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
            JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
            ON a.product_id = b.id
            WHERE status = 'Complete' AND EXTRACT(year FROM created_at) = 2022
            AND EXTRACT(QUARTER FROM created_at) BETWEEN 1 AND 3
            GROUP BY category, year,month,quartile_month))
WHERE rnk = 1
ORDER BY month ASC;
''').to_dataframe()

In [10]:
# Menampilkan data_jumlah_category
data_jumlah_category

,year,month,quartile_month,jenis_category,jumlah_category
0,2022,1,1,Intimates,81
1,2022,2,1,Tops & Tees,82
2,2022,3,1,Intimates,119
3,2022,4,2,Jeans,97
4,2022,5,2,Intimates,111
5,2022,6,2,Jeans,108
6,2022,7,3,Jeans,110
7,2022,8,3,Intimates,121
8,2022,9,3,Intimates,138


Inisght :

- Dari data diatas, diketahui bahwa kategori `Intimates` menduduki posisi teratas disusul oleh `Jeans` dari 9 bulan dalam kategori dengan jumlah terbanyak. Selain itu, jumlah tertinggi berada pada Bulan September

**5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!**

In [11]:
# Memasukan data_total_harga_barang menggunakan query SQL
data_total_harga_barang = client.query('''
SELECT year, month,quartile_month, jenis_category, revenue, jumlah_category
FROM(
      SELECT year, month, quartile_month,jenis_category,
      RANK() OVER(PARTITION BY month ORDER BY revenue DESC) AS rnk, revenue, jumlah_category
      FROM( SELECT category AS jenis_category, 
            EXTRACT(year FROM created_at) AS year,
            EXTRACT(month FROM created_at) AS month,
            EXTRACT(QUARTER FROM created_at) AS quartile_month, 
            SUM(retail_price)-SUM(cost) AS revenue,
            COUNT(category) AS jumlah_category
            FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
            JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
            ON a.product_id = b.id
            WHERE status = 'Complete' AND EXTRACT(year FROM created_at) = 2022
            AND EXTRACT(QUARTER FROM created_at) BETWEEN 1 AND 3
            GROUP BY category, year,month,quartile_month))
WHERE rnk = 1
ORDER BY month ASC;
''').to_dataframe()

In [12]:
# Menampilkan data_total_harga_barang
data_total_harga_barang

,year,month,quartile_month,jenis_category,revenue,jumlah_category
0,2022,1,1,Outerwear & Coats,5879.979812,60
1,2022,2,1,Outerwear & Coats,3507.235525,41
2,2022,3,1,Outerwear & Coats,5454.062446,68
3,2022,4,2,Outerwear & Coats,4888.132240,57
4,2022,5,2,Outerwear & Coats,5281.724055,64
5,2022,6,2,Outerwear & Coats,6933.586816,69
6,2022,7,3,Outerwear & Coats,6206.785251,71
7,2022,8,3,Outerwear & Coats,7196.936794,79
8,2022,9,3,Jeans,6175.411399,129


Insight :

- Dari data diatas, diketahui bahwa `Outerwear & Coats` menduduki peringkat teratas berdasarkan nilai revenue

- Diketahui juga bahwa jumlah transaksi tidak berkorelasi dengan nilai revenue sehingga yang tadinya `Intimates` menduduki peringkat teratas berdasarkan jumlah transaksi digantikan oleh `Outerwear & Coats` berdasarkan nilai revenue. Artinya jumlah transaksi belum tentu memberikan nilai revenue paling tinggi

**6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)**

In [13]:
# Memasukan data_korelasi menggunakan query SQL
data_korelasi = client.query('''
SELECT 
  CORR(jumlah_user, jumlah_transaksi) AS korelasi_transaksi_user,
  CORR(jumlah_user, total_revenue) AS korelasi_penjualan_user
FROM (
      SELECT
      EXTRACT(year FROM b.created_at) AS year,
      EXTRACT(month FROM b.created_at) AS month,
      EXTRACT(QUARTER FROM b.created_at) AS quartile_month, 
      COUNT(DISTINCT a.user_id) AS jumlah_user,
      COUNT(DISTINCT a.order_id) AS jumlah_transaksi,
      SUM(sale_price) AS total_revenue
      FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
      JOIN `bigquery-public-data.thelook_ecommerce.orders` AS b
      ON a.order_id = b.order_id
      WHERE b.status = 'Complete'AND EXTRACT(year FROM b.created_at) =2022
      AND EXTRACT(QUARTER FROM b.created_at) BETWEEN 1 AND 3
      GROUP BY year, month, quartile_month)
''').to_dataframe()

In [14]:
# Menampilkan data_korelasi
data_korelasi

,korelasi_transaksi_user,korelasi_penjualan_user
0,0.999909,0.97271


Insight :

- Dari data diatas diketahui bahwa korelasi antara transaksi dan user serta penjualan dan user yaitu saling berkorelasi karena nilainya mendekati angka 1

## **IV. Poin Analisis**

- Dari data diatas diketahui bahwa terjadi peningkatan dari jumlah transaksi, jumlah user, dan total revenue yang dilakukan dari Bulan Januari - September tahun 2022

- Berdasarkan jumlah kategori, product `intimates` menjadi product favorit untuk dibeli sedangkan berdasarkan total revenue product `outerwear & coats` menjadi product paling memberikan profit

- Dapat ditinjau bahwa adanya korelasi antara jumlah transaksi dengan jumlah user dan total revenue dengan jumlah user

## **V. Jawaban Pertanyaan**

**1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!**

---

Iya dapat terukur ditinjau dari peforma jumlah transaksi, jumlah user, dan total pendapatan yang tiap bulannya rata-rata mengalami peningkatan

**2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?**

---

Tidak masuk akal, peningkatan pendapatan pada tiap kuartal rata-rata sebesar `3,000 USD`

**3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!**

---

Pada akhir kuartal 3 diketahui bahwa jumlah total revenue yaitu `379,994.63 USD` sehingga perusahaan The Look menurut saya layak untuk mendapatkan investor baru karena prospek pada kuartal 4 cukup menjanjikan